In [1]:
from collections import Counter
from itertools import product
import numpy as np
import pandas as pd
import pickle
import re
from scipy import spatial


In [2]:
vec_locs = sorted(["".join(x) for x in product("YMN", repeat = 5)])
vec_locs[:5]

['MMMMM', 'MMMMN', 'MMMMY', 'MMMNM', 'MMMNN']

In [3]:
text1 = "Y"
text2 = "M"
text3 = "N"
text = text1+text2+text3
text.replace("Y", "y").replace("🟩", "Y".replace("M", "m").replace("🟨", "M").replace("N", "n").replace("⬛", "N").replace("⬜", "N"))
guesses = re.findall("([YMN]+)", text)
print(text)

YMN


In [4]:
"""if answer[pos]==guess[pos]:
   print(text1)

unmatched_answer_chars = 0
unmatched_guess_chars = 0
this_guess_num = 0 


for i in range(5):
        if answer[i]==guess[pos]:
            if answer[i]!=guess[i]:
                unmatched_answer_chars += 1
        if guess[i]==guess[pos]:
            if answer[i]!=guess[i]:
                unmatched_guess_chars += 1
                if i<pos:
                    this_guess_num += 1

print(unmatched_answer_chars)
"""

'if answer[pos]==guess[pos]:\n   print(text1)\n\nunmatched_answer_chars = 0\nunmatched_guess_chars = 0\nthis_guess_num = 0 \n\n\nfor i in range(5):\n        if answer[i]==guess[pos]:\n            if answer[i]!=guess[i]:\n                unmatched_answer_chars += 1\n        if guess[i]==guess[pos]:\n            if answer[i]!=guess[i]:\n                unmatched_guess_chars += 1\n                if i<pos:\n                    this_guess_num += 1\n\nprint(unmatched_answer_chars)\n'

In [5]:

# Extract the Wordle result from a tweet, and remap the unicode feedback to Y/M/N
def wordle_guesses(tweet):
    text = (tweet.replace("Y", "y").replace("🟩", "Y")
                 .replace("M", "m").replace("🟨", "M")
                 .replace("N", "n").replace("⬛", "N").replace("⬜", "N"))
    guesses = re.findall("([YMN]+)", text)
    return guesses

def evaluate_guess_char(answer, guess, pos):
    if answer[pos]==guess[pos]:
        return "Y"
    unmatched_answer_chars = 0
    unmatched_guess_chars = 0
    this_guess_num = 0 
    for i in range(5):
        if answer[i]==guess[pos]:
            if answer[i]!=guess[i]:
                unmatched_answer_chars += 1
        if guess[i]==guess[pos]:
            if answer[i]!=guess[i]:
                unmatched_guess_chars += 1
                if i<pos:
                    this_guess_num += 1
    if this_guess_num<unmatched_answer_chars:
        return "M"
    return "N"

def evaluate_guess(answer, guess):
    return "".join(evaluate_guess_char(answer, guess, i) for i in range(5))

assert evaluate_guess("tools", "break")=="NNNNN"
assert evaluate_guess("tools", "tools")=="YYYYY"
assert evaluate_guess("tools", "tolls")=="YYNYY"
assert evaluate_guess("tools", "books")=="NYYNY"
assert evaluate_guess("tools", "broke")=="NNYNN"
assert evaluate_guess("tools", "yahoo")=="NNNMM"
assert evaluate_guess("tools", "bongo")=="NYNNM"
assert evaluate_guess("tools", "brook")=="NNYMN"
assert evaluate_guess("rates", "ranks")=="YYNNY"
assert evaluate_guess("rates", "apple")=="MNNNM"

In [6]:
with open("results/vec_all.pickle", "rb") as f:
    v_all = pickle.load(f)

with open("results/vec_ratio.pickle", "rb") as f:
    v_ratio = pickle.load(f)

with open("results/invalid_results.pickle", "rb") as f:
    invalid_results = pickle.load(f)
    
with open("results/answers.txt", "r") as f:
    answers = f.read().split("\n")

with open("results/other_words.txt", "r") as f:
    other_words = f.read().split("\n")

words = answers + other_words
for answer in ["agora", "pupal", "lynch", "harry", "fibre", "slave", "hasty", "fella"]: # NYTimes version skipped these
    answers.remove(answer)
    
# NYTimes version replaced these
answers[324] = "shiny"
answers[325] = "gecko"

In [7]:
df = pd.read_csv("updated/tweets.csv")
df

wordle_id             tweet_id                 tweet_date  \
0             210  1482553374591660037  2022-01-16 03:20:43+00:00   
1             210  1482553387937898499  2022-01-16 03:20:46+00:00   
2             210  1482553422276698113  2022-01-16 03:20:55+00:00   
3             210  1482553436910628866  2022-01-16 03:20:58+00:00   
4             210  1482553445726908420  2022-01-16 03:21:00+00:00   
...           ...                  ...                        ...   
589258        353  1534172597897900039  2022-06-07 13:57:05+00:00   
589259        353  1534172609088208896  2022-06-07 13:57:08+00:00   
589260        353  1534172616600145920  2022-06-07 13:57:10+00:00   
589261        353  1534172619892662272  2022-06-07 13:57:10+00:00   
589262        353  1534172624351158272  2022-06-07 13:57:11+00:00   

             tweet_username                                         tweet_text  
0                  bpszebes       Wordle 210 4/6\n\n⬛⬛🟨🟨⬛\n🟩⬛⬛⬛⬛\n🟩🟩🟨⬛⬛\n🟩🟩🟩🟩🟩  
1                cruisecoup       Wordle 210 4/6\n\n⬜⬜⬜⬜⬜\n🟩🟩🟨🟨⬜\n🟩🟩⬜🟩🟨\n🟩🟩🟩🟩🟩  
2                DestroVega       Wordle 210 4/6\n\n⬜⬜⬜🟨⬜\n⬜🟩⬜🟨⬜\n⬜🟩⬜🟨⬜\n🟩🟩🟩🟩🟩  
3               brenmardash              Wordle 210 3/6\n\n⬜⬜🟨⬜⬜\n🟨🟨⬜⬜⬜\n🟩🟩🟩🟩🟩  
4               KatieHowse2              Wordle 210 3/6\n\n⬛⬛🟨⬛⬛\n🟩🟩🟩⬛⬛\n🟩🟩🟩🟩🟩  
...                     ...                                                ...  
589258            498500844  Wordle 353 5/6\n\n🟨🟨⬜⬜⬜\n⬜⬜🟨⬜🟨\n⬜🟩🟩⬜⬜\n⬜🟩🟩🟩⬜\n...  
589259  1520593678712115200       Wordle 353 4/6\n\n🟩🟩⬜⬜⬜\n🟩🟩🟩⬜⬜\n🟩🟩🟩⬜⬜\n🟩🟩🟩🟩🟩  
589260             75208020              Wordle 353 3/6\n\n⬛⬛⬛⬛⬛\n⬛🟨🟨⬛⬛\n🟩🟩🟩🟩🟩  
589261             14467116              Wordle 353 3/6\n\n⬛⬛⬛⬛⬛\n🟩🟨⬛⬛🟩\n🟩🟩🟩🟩🟩  
589262            264376328              Wordle 353 3/6\n\n🟩⬜⬜⬜🟩\n🟩🟨⬜⬜🟩\n🟩🟩🟩🟩🟩  

[589263 rows x 5 columns]

In [ ]:
results = []

for i in sorted(set(df["wordle_id"])):
    #i = 351
    tweet_texts = df[df["wordle_id"]==i]["tweet_text"]
    
    games = [wordle_guesses(tweet) for tweet in tweet_texts]   
    
    all_counts = Counter(res for game in games for res in game if len(game)>=2 and game[-1]=="YYYYY")
    first_counts = Counter(game[0] for game in games if len(game)>=2 and game[-1]=="YYYYY")
    penultimate_counts = Counter(game[-2] for game in games if len(game)>=2 and game[-1]=="YYYYY")
    
    vec_all = [all_counts[res] for res in vec_locs]
    vec_first = [first_counts[res] for res in vec_locs]
    vec_penultimate = [penultimate_counts[res] for res in vec_locs]
    vec_ratio = [penultimate_counts[res]/(all_counts[res]+1e-6) for res in vec_locs]
    
    dists = {"all": [], "ratio": []}
    
    for key in v_all:
        dists["all"].append((spatial.distance.cosine(vec_all, v_all[key]), key))
        dists["ratio"].append((spatial.distance.cosine(vec_ratio, v_ratio[key]), key))
    
    dists["invalid"] = [(len([g for g in games if any(r in invalid_results[key] for r in g)]), key) for key in v_all]

    ranks = {}
    for d in dists:
        s = sorted(dists[d])
        ranks[d] = {s[i][1]: i for i in range(len(s))}
    
    overall = []
    for key in v_all:
        overall.append((sum(ranks[r][key] for r in ranks), key))
    
    my_guess = sorted(overall)[0][1]
    answer = answers[i]
    answer_rank = [x[1] for x in sorted(overall)].index(answer)
        
    results.append([i, my_guess, answer, answer_rank, overall, dists, ranks])
    feedback = evaluate_guess(answer, my_guess).replace("Y", "🟩").replace("M", "🟨").replace("N", "⬛")

    if answer==my_guess:
        feedback += " !!!!!!"
    else:
        feedback += " :(:( the actual answer ranked %d on my guess list" % (answer_rank)
    
    if i<max(df["wordle_id"]):
        print("For Wordle %d, my guess was %s. %s" % (i, my_guess, feedback))
    else:
        # Hide today's answer
        print("For today's Wordle %d, my result was: %s" % (i, feedback))

For Wordle 210, my guess was panic. 🟩🟩🟩🟩🟩 !!!!!!
For Wordle 211, my guess was solar. 🟩🟩🟩🟩🟩 !!!!!!
For Wordle 212, my guess was shire. 🟩🟩🟩🟩🟩 !!!!!!
For Wordle 213, my guess was proxy. 🟩🟩🟩🟩🟩 !!!!!!
For Wordle 214, my guess was point. 🟩🟩🟩🟩🟩 !!!!!!
For Wordle 215, my guess was robot. 🟩🟩🟩🟩🟩 !!!!!!
For Wordle 216, my guess was prick. 🟩🟩🟩🟩🟩 !!!!!!
For Wordle 217, my guess was wince. 🟩🟩🟩🟩🟩 !!!!!!
For Wordle 218, my guess was crimp. 🟩🟩🟩🟩🟩 !!!!!!
For Wordle 219, my guess was knoll. 🟩🟩🟩🟩🟩 !!!!!!
For Wordle 220, my guess was sugar. 🟩🟩🟩🟩🟩 !!!!!!
For Wordle 221, my guess was whack. 🟩🟩🟩🟩🟩 !!!!!!
For Wordle 222, my guess was mount. 🟩🟩🟩🟩🟩 !!!!!!
For Wordle 223, my guess was perky. 🟩🟩🟩🟩🟩 !!!!!!
For Wordle 224, my guess was could. 🟩🟩🟩🟩🟩 !!!!!!
For Wordle 225, my guess was wrung. 🟩🟩🟩🟩🟩 !!!!!!
For Wordle 226, my guess was light. 🟩🟩🟩🟩🟩 !!!!!!
For Wordle 227, my guess was those. 🟩🟩🟩🟩🟩 !!!!!!
For Wordle 228, my guess was moist. 🟩🟩🟩🟩🟩 !!!!!!
For Wordle 229, my guess was shard. 🟩🟩🟩🟩🟩 !!!!!!
For Wordle 230, my g

In [ ]:
# Unhide the output of this cell to reveal today's guess
print("For Wordle %d, my guess was %s. %s" % (i, my_guess, feedback))